In [1]:
# imports
import os
import configparser
import datetime
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DateType
from pyspark.sql.functions import udf, col

In [2]:
config = configparser.ConfigParser()
config.read('config.cfg')

#input_dir
input_dir = config.get('INPUT', 'INPUT_DIR')

# staging_prefix
staging_dir = config.get('OUTPUT', 'STAGING_PREFIX')
analytics_dir = config.get('OUTPUT', 'ANALYTICS_PREFIX')

# set AWS env variable
os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS', 'AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS', 'AWS_SECRET_ACCESS_KEY')

# raw data path
raw_data_path = config.get('INPUT', 'RAW_DATA_PATH')

In [3]:
def create_spark():
    spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0")\
    .enableHiveSupport().getOrCreate()
    return spark

In [4]:
spark = create_spark()

In [5]:
immigrations = spark.read.parquet('immigrations/')
immigrants = spark.read.parquet('immigrants/')
us_temp = spark.read.parquet('temperature/')
us_states = spark.read.parquet('states/')
us_cities = spark.read.parquet('cities/')
airports = spark.read.parquet('airports/')
time = spark.read.parquet('time/')

In [6]:
immigrations.createOrReplaceTempView('immigrations')

In [9]:
time.createOrReplaceTempView('Time')

In [23]:
assert len(immigrations.where(col('id').isNull()).collect()) == 0

In [25]:
assert us_states.count() == 49

True

In [29]:
assert time.agg({"year": "max"}).collect()[0]["max(year)"] == 2016

In [32]:
assert us_temp.agg({"year": "max"}).collect()[0]["max(year)"] == 2013